In [41]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import statistics as st
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from scipy.stats import shapiro 
from scipy.stats import lognorm
from scipy.stats import kstest
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 





red_wines = pd.read_csv("winequality-red.csv", sep = ";")
white_wines = pd.read_csv("winequality-white.csv", sep =";")

In [49]:
# formatting
red_wines.columns= red_wines.columns.str.replace(' ','_')
white_wines.columns = white_wines.columns.str.replace(' ','_')

#Quality categories
red_wines ['quality_label'] = red_wines['quality'].apply(lambda value: 'low' if value <= 5 
                                                        else 'medium' if value <= 7 
                                                        else 'high')

red_wines['quality_label'] = pd.Categorical(red_wines['quality_label'],
categories=['low', 'medium', 'high'])

white_wines ['quality_label'] = white_wines['quality'].apply(lambda value: 'low' if value <= 5 
                                                        else 'medium' if value <= 7 
                                                        else 'high')

white_wines["quality_label"] = pd.Categorical(white_wines["quality_label"], categories = ["low","medium","high"])

# Type of wine categories
red_wines ["type"] = 'Red Wine'
red_wines['type'] = pd.Categorical(red_wines['type'],
categories=["Red Wine","White Wine"])

white_wines ["type"] = 'White Wine'
white_wines['type'] = pd.Categorical(white_wines['type'],
categories=["Red Wine","White Wine"])

# merge datasets
white_and_red = pd.merge(red_wines, white_wines, how = "outer")

#make copy
white_and_red_ml= white_and_red.copy()

In [4]:
white_and_red_ml["quality_label"].head(10)

0    medium
1      high
2    medium
3       low
4      high
5    medium
6    medium
7       low
8       low
9       low
Name: quality_label, dtype: category
Categories (3, object): ['low', 'medium', 'high']

# Filtering outliers

In [52]:
white_wines_clean= white_wines.drop("quality", axis=1)
numeric_columns_white = white_wines_clean.select_dtypes(include=['float64', 'int64'])

white_wine_filtered= white_wines.copy()
for column in numeric_columns_white.columns:
        q1 = np.quantile(numeric_columns_white[column], 0.25)
        q3 = np.quantile(numeric_columns_white[column], 0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        # Filter rows based on the column's outlier range
        white_wine_filtered = white_wine_filtered[(white_wine_filtered[column] >= lower) & (white_wine_filtered[column] <= upper)]
print("white wine shape; ", white_wines.shape, "\nfiltered white wine shape: ", white_wine_filtered.shape)


white wine shape;  (4898, 14) 
filtered white wine shape:  (4015, 14)


In [56]:
red_wines_clean= red_wines.drop("quality", axis=1)
numeric_columns_red = red_wines_clean.select_dtypes(include=['float64', 'int64'])

red_wine_filtered= red_wines.copy()
for column in numeric_columns_red.columns:
        q1 = np.quantile(numeric_columns_red[column], 0.25)
        q3 = np.quantile(numeric_columns_red[column], 0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        # Filter rows based on the column's outlier range
        red_wine_filtered = red_wine_filtered[(red_wine_filtered[column] >= lower) & (red_wine_filtered[column] <= upper)]
print("red wine shape; ", red_wines.shape, "\nfiltered red wine shape: ", red_wine_filtered.shape)

red wine shape;  (1599, 14) 
filtered red wine shape:  (1194, 14)


# Encoding

In [ ]:
le = LabelEncoder()
white_and_red_ml['quality_label'] = le.fit_transform(white_and_red_ml['quality_label'])
white_and_red_ml["quality_label"].head()



0    2
1    0
2    2
3    1
4    0
Name: quality_label, dtype: int64

In [7]:
# split data

X=white_and_red_ml.drop(["type","quality_label","quality"], axis=1)
y= white_and_red_ml["quality_label"]

print("original:", white_and_red_ml.shape, "\nX:", X.shape, "\ny:", y.shape)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

original: (6497, 14) 
X: (6497, 11) 
y: (6497,)


In [ ]:
# Standarization and normalization

norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform testing data
X_test_norm = norm.transform(X_test)

In [45]:
## logistic regression
LR_model = LogisticRegression(max_iter=700)
LR_model.fit(X_train_norm, y_train)
preds = LR_model.predict(X_test_norm)

## Accuracy
acc = accuracy_score(y_test, preds)
print(acc)

0.7038461538461539


In [ ]:
# merge datasets
white_and_red_filtered = pd.merge(red_wine_filtered, white_wine_filtered, how = "outer")

le = LabelEncoder()

white_and_red_filtered ['quality_label'] = le.fit_transform(white_and_red_filtered['quality_label'])

from sklearn.model_selection import train_test_split
X2=white_and_red_filtered.drop(["type","quality_label","quality"], axis=1)
y2= white_and_red_filtered["quality_label"]

print("original:", white_and_red_filtered .shape, "\nX:", X2.shape, "\ny:", y2.shape)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.20, random_state=42)
## standarization
norm = MinMaxScaler().fit(X2_train)

# transform training data
X2_train_norm = norm.transform(X2_train)

# transform testing data
X2_test_norm = norm.transform(X2_test)
## logistic regression
LR_model2 = LogisticRegression(max_iter=700)
LR_model2.fit(X2_train_norm, y2_train)
preds2 = LR_model2.predict(X2_test_norm)

## Accuracy
acc2 = accuracy_score(y2_test, preds2)

print(acc2)


original: (5209, 14) 
X: (5209, 11) 
y: (5209,)
0.7092130518234165


LogisticRegression(max_iter=700)

In [61]:
# define model
RF_clf = RandomForestClassifier(random_state=42)
# fit model
RF_clf.fit(X2_train, y2_train)
# make predictions
RF_preds = RF_clf.predict(X2_test)

RF_acc = accuracy_score(y2_test, RF_preds)
print( "Random Forest: ", RF_acc)

Random Forest:  0.817658349328215
